In [1]:
from __future__ import division
import cv2
import time
import sys
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import pickle
import keras

from keras.layers import Input, GlobalAveragePooling2D, TimeDistributed, Dense, Convolution2D, Flatten, CuDNNLSTM, LSTM, Dropout, Reshape
from keras.models import Model
from keras.optimizers import Nadam

### Load data

In [2]:
folder_path = os.path.join(os.getcwd(), "dataset")
print(folder_path)

D:\Python\dataset


In [10]:
folder_path = "D:\\Python\\dataset\\"
test_path = "resolution_64x64x1_count_24_test_data.pickle"
emotions_count = 8

with open(folder_path + test_path, 'rb') as handle:
    test_data = pickle.load(handle)

x_test = []
y_test = []
for item in test_data:
    gray_arr = []
    for image in list(item["images"]): 
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        gray_arr.append(image)
    
    gray_arr = np.asarray(gray_arr)
    x_test.append(np.expand_dims(gray_arr,axis=2))
    y_test.append(item["emotion"])

# x_test = []
# y_test = []
# for item in test_data:
#     x_test.append(item["images"])
#     y_test.append(item["emotion"])
    
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

print(x_test.shape)
print(y_test.shape)

(192, 30, 64, 1, 64, 3)
(192, 8)


### Load model

In [12]:
model_path = "models\\base_emotion_classification_model.hdf5"
model = load_model(model_path, compile=False) 

lr = 0.005
sgd = keras.optimizers.SGD(lr=lr, momentum=0.9, nesterov=True, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

### Test on all data

In [13]:
result = model.evaluate(x_test, y_test, batch_size=1)
print(result)

ValueError: in user code:

    File "C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1820, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1804, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1792, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1756, in test_step
        y_pred = self(x, training=False)
    File "C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\HP\AppData\Local\Prog

### Test emotions separately

In [ ]:
results_x = []
results_y = []
results = []
emotions_count = 8
for j in range(0, emotions_count):
    emotion_xtest_array = []
    emotion_ytest_array = []
    for i in range(0, len(y_test)):
        if(np.argmax(y_test[i]) == j):
            emotion_xtest_array.append(x_test[i])
            emotion_ytest_array.append(y_test[i])

    emotion_xtest_array = np.asarray(emotion_xtest_array)
    emotion_ytest_array = np.asarray(emotion_ytest_array)
    result = model.evaluate(emotion_xtest_array, emotion_ytest_array, batch_size=1)
    results_x.append(result[0])
    results_y.append(result[1]*100)
    results.append(result)

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

emotions = ["нейтральная", "спокойствие"," счастье", "грусть", "злость", "испуг", "отвращение", "удивление"]
y_pos = np.arange(len(emotions))

plt.bar(y_pos, results_y, align='center', alpha=0.5)
plt.xticks(y_pos, emotions, rotation = "45")
plt.ylabel('Точность, %')
plt.title('Точность распознавания каждой эмоции')

plt.show()

In [ ]:
target_names = ['нейтральная', 'спокойствие','счастье', 'грусть', 'злость', 'испуг', 'отвращение', 'удивление']

Y_pred = model.predict(x_test)
y_pred_confusion = np.argmax(Y_pred, axis=1)
y_test_confusion = np.argmax(y_test, axis=1)

In [ ]:
from sklearn import svm, datasets
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Нормализованная матрица неточностей'
        else:
            title = 'Матрица неточностей без нормализации'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Нормализованная матрица неточностей")
    else:
        print('Матрица неточностей без нормализации')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='Экспертные решения',
           xlabel='Решения классификатора')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

In [ ]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test_confusion, y_pred_confusion, classes=target_names,
                      title='Матрица неточностей без нормализации')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test_confusion, y_pred_confusion, classes=target_names, normalize=True,
                      title='Нормализованная матрица неточностей')

plt.show()

In [ ]:
y = classification_report(y_test_confusion, y_pred_confusion, target_names=target_names)

In [ ]:
model.summary()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)